# Link prediction competition

Carlos SAiz Hernández, [CSH](csh1001@alu.ubu.es)

## Librerías

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Variables Globales

In [ ]:
# Hubicacion de los archivos
GRAFOGEXF='/kaggle/input/link-prediction-for-social-networks-2024/social_network_training_1.gexf'
TEST='/kaggle/input/link-prediction-for-social-networks-2024/test_1.csv'

In [ ]:
# Input data files are available in the read-only "../input/" directory
def creacionGrafoMedianteArchivo(file_path):
    """*+
    Crea un grafo a partir de un archivo gexf

    Args:
        file_path (srt): Link del archivo gexf

    Returns:
        nx.Graph: Grafo creado a partir del archivo gexf
    """""
    G = nx.read_gexf(file_path)
    return G

## Manipulacion de archivos CSV

In [ ]:
def leemosCSV(path: str) -> pd.DataFrame:
    """*+
    Lee un archivo csv y lo convierte en un DataFrame

    Args:
        path (str): Link del archivo csv

    Returns:
        pd.DataFrame : DataFrame creado a partir del archivo csv
    """
    test=pd.read_csv(path)
    
    test['Predicted']=False
    
    return test

def obtenemosPares(test: pd.DataFrame) -> list:
    """*+
    Obtiene los pares de nodos de un DataFrame

    Args:
        test (pd.DataFrame): DataFrame con los pares de nodos

    Returns:
        list: Lista con los pares de nodos
    """
    listaTest = test.values.tolist()
    
    resultado = [[n for n in item[0].split('-')]for item in listaTest]
    
    return resultado

def escrituraArchivo(diccionario: dict):
    """*+
    Escribe un archivo csv con los datos de un diccionario

    Args:
        diccionario (dict): Diccionario con los datos a escribir
    """
    diccionarioEstructurado = estructuracionEscritura(diccionario)
    
    df = pd.DataFrame(diccionarioEstructurado)
    
    df.to_csv('submission.csv', index=False)
    

def estructuracionEscritura(diccionario: dict) -> dict:
    """*+
    Estructuramos un diccionario para ser escrito en un archivo csv
    
    Args:
        diccionario (dict): Diccionario con los datos a escribir

    Returns:
        dict: Diccionario estructurado para ser escrito en un archivo 
            csv con keys 'Id' y 'Predicted'
    """
    keys = reestructuracionKeys(list(diccionario.keys()))
    Values = list(diccionario.values())
    
    resultado = {'Id': keys,
                 'Predicted': Values       
    }
    return resultado
    
def reestructuracionKeys(lista: list) -> list:
    """*+
    Reestructuramos una lista de pares de nodos para ser escrita en un archivo csv
    Args:
        lista (list): Lista con los pares de nodos

    Returns:
        list: Lista reestructurada para ser escrita en un archivo csv
    """
    return [f'{par[0]}-{par[1]}' for par in lista]
    

## Manipulacion de listas

In [ ]:
def filtradoDeLista(lista: list, diccionario: dict) -> list:
    """*+
    Realiza un filtrado de una lista de pares de nodos con un diccionario

    Args:
        lista (list): Lista con los pares de nodos
        diccionario (dict): Diccionario con los nodos del grafo

    Returns:
        list: Lista con los pares de nodos filtrados
    """
    listaResultado=[]
    
    for pares in lista:
        if pares[0] in diccionario.keys() and pares[1] in diccionario.keys():
            listaResultado.append((pares[0],pares[1]))
    return listaResultado

## Algoritmo de predicción

In [ ]:
def prediccionDeLinks(pares: list, G: nx.Graph) -> dict:
    """*+
    Realiza la predicción de links de un grafo usando el algoritmo adamic adar

    Args:
        pares (list): Lista con los pares de nodos
        G (nx.Graph): Grafo a analizar

    Returns:
        dict: Diccionario con los pares de nodos y su predicción ordenados de forma descendente
            siendo los primeros 1000 los que se consideran True y los demás False
    """
    
    adamicAdarScore = [((par1, par2),s) for par1, par2,  s in nx.adamic_adar_index(G, pares)]
    print('\t->Coeficientes calculados')
    
    adamicAdarScore.sort(key=lambda x: x[1], reverse=True)
    print('\t->Nodos ordenados según su coeficiente de forma descendente')
    
    links = {pair: True for pair, _ in adamicAdarScore[:1000]}
    links.update({pair: False for pair, _ in adamicAdarScore[1000:]})
    print('\t->Nodos estructurados')
    return links

In [ ]:
G=creacionGrafoMedianteArchivo(GRAFOGEXF)
print('Nodes: %i, edges: %i' %(G.number_of_nodes(),G.number_of_edges()))

# Dibujamos el grafo
fig = plt.figure(figsize=(7,7))
nx.draw(G,pos=nx.kamada_kawai_layout(G),node_color='black',
        alpha=0.5,node_size=10,edge_color='orange',width=0.5)

# Guardamos los nodos y sus vecinos como dicionario {nodo:[vecinos]}
diccionarioGrafo = {nodo: list(G.neighbors(nodo)) for nodo in G.nodes}

test = leemosCSV(TEST)

# [[nodo1, nodo2, valor],...]
listaParesCSV = obtenemosPares(test)

listaDepurada = filtradoDeLista(listaParesCSV, diccionarioGrafo)

print('Inicia la predicción, esto puede tardar un rato')
prediccion = prediccionDeLinks(listaDepurada, G)

print('Predicción finalizada')
escrituraArchivo(prediccion)
